In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import rioxarray as rxr
from glob import glob
from pathlib import Path
from skimage.transform import resize

In [ ]:
# Dialation of the clouds

In [ ]:
data_regex = '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/tiles_cas/*.tif'
#model_filename = '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/model/25-32.46.hdf5'
#model_filename = '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/model/24-31.54.hdf5'
model_filename = '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/model/22-27.97.hdf5'

In [ ]:
model = tf.keras.models.load_model(model_filename)
model.summary()

In [ ]:
data_list = glob(data_regex)
len(data_list)

In [ ]:
data_array = []
label_array = []

for filename in data_list:
    
    data = rxr.open_rasterio(filename).values
    
    if data.min() < -100:
        continue
    
    data = np.moveaxis(data, 0, -1)
    data = resize(data, (128, 128))
    data = data / 10000.0
    data_array.append(data)

    label_array.append(float((Path(filename).stem).split('_')[-1]))

data_array = np.array(data_array)
label_array = np.array(label_array)
data_array.shape, label_array.shape

In [ ]:
predictions = model.predict(data_array, batch_size=128)
predictions = np.squeeze(predictions)
predictions.shape

In [ ]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(label_array, predictions), 4)) 
print("Mean absolute percent error =", round(sm.mean_absolute_percentage_error(label_array, predictions), 4)) 
print("Mean squared error =", round(sm.mean_squared_error(label_array, predictions), 4)) 
print("Median absolute error =", round(sm.median_absolute_error(label_array, predictions), 4)) 
print("Explain variance score =", round(sm.explained_variance_score(label_array, predictions), 4)) 
print("R2 score =", round(sm.r2_score(label_array, predictions), 4))

In [ ]:
for i in range(data_array.shape[0]):
    
    print(f'Label: {label_array[i]}, Prediction: {predictions[i]}')
    plt.imshow(data_array[i, :, :, 3])
    plt.show()